In [1]:
## Elaboração Desenvolvimento de Controle de Qualidade para Questionários da Área da Saúde 

In [2]:
#Protótipo de Elaboração : Sistema de Controle de Qualidade para Questionários em Estudos de Saúde e Epidemiologia
!pip install openai
!pip install google-cloud-speech
!pip install pydub

In [3]:
from pydub import AudioSegment

C:\Users\ben10\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
# ETAPA 1: IMPORTAÇÃO DE BIBLIOTECAS
import os
import csv
import openai
from google.cloud import speech
from pydub import AudioSegment
import pandas as pd

In [5]:
# ETAPA 2: CONFIGURAÇÕES INICIAIS

# Caminho para o arquivo de credenciais da Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\ben10\\Downloads\\prototipo-de-elaboracao-1be26843005f.json"

# Chave da API OpenAI
#openai.api_key = "sua_openai_key"

# Nome do arquivo de áudio e CSV
#AUDIO_FILE = "exemplo_audio.wav"
#PERGUNTAS_CSV = "perguntas.csv"

#você já pode usar pydub para converter arquivos para .wav, como por exemplo:
#audio = AudioSegment.from_mp3("seuarquivo.mp3")
#audio.export("convertido.wav", format="wav")

In [ ]:
# ETAPA 3: LEITURA DAS PERGUNTAS

df_perguntas = pd.read_csv(PERGUNTAS_CSV)
print("Perguntas carregadas:")
print(df_perguntas)

In [ ]:
# ETAPA 4: TRANSCRIÇÃO COM GOOGLE CLOUD SPEECH-TO-TEXT

def transcreve_audio_google(audio_path):
    client = speech.SpeechClient()

    with open(audio_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,  # ou conforme o arquivo
        language_code="pt-BR",
    )

    response = client.recognize(config=config, audio=audio)

    transcript = " ".join([result.alternatives[0].transcript for result in response.results])
    return transcript

#transcricao = transcreve_audio_google(AUDIO_FILE)
#print("Transcrição do áudio:")
#print(transcricao)

In [ ]:
# ETAPA 5: ANÁLISE COM LLM GPT-4o (OpenAI)

def organiza_respostas(transcricao, perguntas):
    prompt = f"""
    Abaixo está a transcrição de uma entrevista em português, seguida de uma lista de perguntas realizadas na entrevista.

    Transcrição:
    {transcricao}

    Perguntas:
    {', '.join(perguntas)}

    Sua tarefa é organizar as respostas correspondentes a cada pergunta de forma clara, mesmo que a ordem esteja diferente.
    Para cada pergunta, extraia a resposta correspondente da transcrição e escreva no formato:
    
    Pergunta: ...
    Resposta: ...
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=1500
    )

    return response.choices[0].message['content']

resposta_final = organiza_respostas(transcricao, df_perguntas['pergunta'].tolist())
print("Resposta organizada:")
print(resposta_final)


In [ ]:
# Supondo que 'audio_data' já foi carregado e 'dialogo_csv_data' contém o conteúdo do CSV
# Ex: dialogos = [{"pergunta_esperada": "Qual o seu nome?", "resposta_esperada": "Meu nome é Bruno."}]

from openai import OpenAI
import os

# ---- Configuração ----
# ATENÇÃO: Nunca deixe sua chave diretamente no código em produção! Use variáveis de ambiente.
# Este é um exemplo para fins de prototipagem.
os.environ['OPENAI_API_KEY'] = 'SUA_CHAVE_DA_OPENAI_AQUI'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\ben10\\Downloads\\prototipo-de-elaboracao-1be26843005f.json" # Exemplo do caminho corrigido

# Inicializa o cliente OpenAI
client_openai = OpenAI()

# Inicializa o cliente Google Cloud Speech-to-Text
from google.cloud import speech_v1p1beta1 as speech
client_google_speech = speech.SpeechClient()

# Caminho para o áudio de teste local (simulação da entrevista)
audio_file_path = "caminho/para/seu/audio_curto_teste.wav" # Certifique-se de usar um caminho real

# Caminho para o arquivo CSV com perguntas e respostas esperadas
csv_file_path = "caminho/para/seu/perguntas_respostas_esperadas.csv" # Certifique-se de usar um caminho real

# ---- Função para carregar áudio (Exemplo simplificado) ----
def load_audio_content(file_path):
    with open(file_path, "rb") as audio_file:
        content = audio_file.read()
    return content

# ---- Função para carregar CSV (Exemplo simplificado) ----
import csv
def load_csv_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

# ---- Função para transcrever áudio com Google Speech-to-Text ----
def transcribe_audio_google(audio_content, sample_rate_hertz=16000, language_code="pt-BR"):
    audio = speech.RecognitionAudio(content=audio_content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16, # Ajuste conforme seu áudio
        sample_rate_hertz=sample_rate_hertz,
        language_code=language_code,
        model="default" # Pode ser "latest_long" ou "phone_call" dependendo do áudio
    )
    try:
        response = client_google_speech.recognize(config=config, audio=audio)
        transcription = ""
        for result in response.results:
            transcription += result.alternatives[0].transcript
        return transcription
    except Exception as e:
        print(f"Erro na transcrição do Google Speech-to-Text: {e}")
        return None

# ---- Função para processar com OpenAI LLM ----
def process_with_openai_llm(prompt_text, model="gpt-4o"):
    try:
        response = client_openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Você é um assistente útil para analisar a coerência entre perguntas e respostas de entrevistas."},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0.7 # Ajuste para controlar a criatividade
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Erro no processamento com OpenAI LLM: {e}")
        return None

# ---- Fluxo Principal do Protótipo ----
def run_prototype():
    print("Iniciando o protótipo...")

    # 1. Carregar áudio local
    audio_content = load_audio_content(audio_file_path)
    if not audio_content:
        print("Não foi possível carregar o arquivo de áudio. Verifique o caminho.")
        return

    # 2. Transcrever áudio
    print("Transcrevendo áudio com Google Speech-to-Text...")
    transcricao_completa = transcribe_audio_google(audio_content)
    if not transcricao_completa:
        print("Transcrição falhou. Encerrando protótipo.")
        return
    print(f"Transcrição completa do áudio: {transcricao_completa}\n")

    # 3. Carregar perguntas e respostas esperadas do CSV
    perguntas_respostas_esperadas = load_csv_data(csv_file_path)
    if not perguntas_respostas_esperadas:
        print("Não foi possível carregar o arquivo CSV. Verifique o caminho ou o formato.")
        return

    resultados_finais = []

    # 4. Loop para sincronização e processamento com LLM
    print("Iniciando loop de sincronização e processamento com OpenAI LLM...")
    for i, par in enumerate(perguntas_respostas_esperadas):
        pergunta_escrita = par.get("pergunta_esperada", "")
        resposta_escrita = par.get("resposta_esperada", "")

        # SIMPLIFICAÇÃO: Lógica de sincronização básica para o protótipo.
        # Em um sistema real, isso envolveria algoritmos de alinhamento de texto/áudio.
        # Aqui, estamos simplesmente assumindo que podemos "identificar" as partes na transcrição completa.
        # Para um protótipo, você pode até passar a transcrição completa e pedir ao LLM para "extrair"
        # a resposta para uma pergunta específica e compará-la com a esperada.
        
        # Exemplo de prompt simplificado para o LLM para simular a sincronização e análise
        prompt_para_llm_para_analise = (
            f"Considere a seguinte pergunta de questionário: '{pergunta_escrita}'\n"
            f"A resposta esperada é: '{resposta_escrita}'\n"
            f"A transcrição completa do áudio é: '{transcricao_completa}'\n\n"
            f"1. Identifique a resposta mais provável para a pergunta '{pergunta_escrita}' dentro da 'transcrição completa'.\n"
            f"2. Compare a resposta identificada na transcrição com a 'resposta esperada'.\n"
            f"3. Avalie a coerência e a fidelidade da resposta identificada. Atribua uma pontuação de 0 a 100 para a qualidade da correspondência.\n"
            f"4. Sugira ajustes na resposta da transcrição para que ela corresponda melhor à resposta esperada, se necessário."
        )

        print(f"\nProcessando Par {i+1}: Pergunta='{pergunta_escrita}' Resposta Esperada='{resposta_escrita}'")
        analise_llm = process_with_openai_llm(prompt_para_llm_para_analise)
        
        resultado_par = {
            "pergunta_escrita": pergunta_escrita,
            "resposta_escrita": resposta_escrita,
            "transcricao_completa_audio": transcricao_completa, # Manter a completa por enquanto
            "analise_llm": analise_llm
        }
        resultados_finais.append(resultado_par)
        print(f"Análise do LLM para o par {i+1}:\n{analise_llm}")

    # 5. Salvar resultados
    import json
    output_json_path = "resultados_prototipo.json"
    with open(output_json_path, "w", encoding='utf-8') as f:
        json.dump(resultados_finais, f, indent=4, ensure_ascii=False)
    print(f"\nProtótipo concluído. Resultados salvos em: {output_json_path}")

# Executar o protótipo
if __name__ == "__main__":
    # CRIE SEUS ARQUIVOS DE ÁUDIO E CSV DE TESTE ANTES DE EXECUTAR!
    # Exemplo de conteúdo para 'perguntas_respostas_esperadas.csv':
    # pergunta_esperada,resposta_esperada
    # Qual o seu nome?,Meu nome é Bruno.
    # Qual a sua idade?,Tenho 25 anos.

    # Certifique-se de que o audio_file_path e csv_file_path apontem para arquivos existentes
    # e que suas chaves de API estejam configuradas.
    run_prototype()